In [37]:
import pymongo
import pandas as pd
import jieba
import numpy as np
import string
from collections import Counter
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Embedding, Flatten
from keras.models import Sequential

In [38]:
# mongodb uri
conn_str = "mongodb://1.15.118.125:27017/"
# 客户端连接
client = pymongo.MongoClient(conn_str, serverSelectionTimeoutMS=5000)
collection_user = client.get_database("Lab").get_collection("Mission")

In [39]:
# 查询 title 与对应的 tags
title_with_tags = {}
for doc in collection_user.find():
    mission_tags = doc.get("missionTags")
    mission_title = doc.get("title")
    title_with_tags[mission_title] = mission_tags

# pands 格式化输出
ser_tag_tags = pd.Series(title_with_tags)
ser_tag_tags.head(len(ser_tag_tags))

学生骨干成长训练营举办暑期专题培训                     [学生工作, 培训会]
新生教师班主任工作研讨会召开                    [教学建设, 研讨会, 迎新]
2022级本科生线上新生家长会                    [师生交流, 会议, 迎新]
2022级研究生新生入学教育活动                       [迎新, 主题教育]
研究生班级新生见面会顺利举行                         [迎新, 班级活动]
                                       ...       
管理学院第二届工商管理学科高端论坛系列活动: 学科建设座谈会        [教学建设, 交流会]
新进老师培训                                [教学建设, 培训会]
学生骨干训练营的结营仪式                           [学生工作, 仪式]
十二月舍导交流                              [师生交流, 校园生活]
中国管理学前沿研究研讨会: 开幕式与主报告             [学术, 仪式, 汇报/报告]
Length: 101, dtype: object

In [40]:
# 忽略词
eng_words = list(string.ascii_lowercase + string.ascii_uppercase)
exclude_words = ['，', '“', '”', '、', '《', '》', '。', ':', '-', '的', '?', ' ', ',', '&']
for w in eng_words:
    exclude_words.append(w)
for i in range(10):
    exclude_words.append(str(i))
print(exclude_words)

['，', '“', '”', '、', '《', '》', '。', ':', '-', '的', '?', ' ', ',', '&', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [41]:
# 分割 title 的词语
total_words = []
title_with_words = {}
for title in title_with_tags.keys():
    for i in exclude_words:
        title = title.replace(i, '')
    title_with_words[title] = []
    # jieba拆分方法
    words = jieba.cut(title, cut_all=False)
    for word in words:
        total_words.append(word)
        title_with_words[title].append(word)

fre = Counter(total_words)
word_fre = sorted(fre.items(), key=lambda i: i[1], reverse=True)
print("总词数: " + len(word_fre).__str__())
print(word_fre)

总词数: 334
[('活动', 25), ('级', 13), ('顺利', 13), ('分享', 13), ('举行', 12), ('暨', 12), ('开展', 11), ('管理', 11), ('主题', 10), ('二十大', 10), ('召开', 9), ('精神', 9), ('学习', 8), ('新生', 7), ('论坛', 7), ('学术', 7), ('会', 7), ('班', 7), ('党', 7), ('学生', 6), ('举办', 6), ('培训', 6), ('入学', 6), ('做', 6), ('班会', 6), ('学院', 6), ('见面会', 5), ('大会', 5), ('年', 5), ('宣讲会', 5), ('竞赛', 5), ('喻园', 5), ('大学', 5), ('党支部', 5), ('月', 5), ('知识', 5), ('骨干', 4), ('专题', 4), ('本科生', 4), ('教育', 4), ('教授', 4), ('商学院', 4), ('师生', 4), ('顺利开展', 4), ('座谈会', 4), ('十一月', 4), ('啦', 4), ('党史', 4), ('德育', 4), ('训练营', 3), ('工作', 3), ('研讨会', 3), ('党员', 3), ('舍导进', 3), ('宿舍', 3), ('有限公司', 3), ('国奖', 3), ('成功', 3), ('杯', 3), ('课程', 3), ('院长', 3), ('工商', 3), ('经验', 3), ('团支部', 3), ('第一次', 3), ('本科', 3), ('党日', 3), ('与', 3), ('贯彻', 3), ('｜', 3), ('仪式', 3), ('项目', 3), ('电脑', 3), ('交流', 3), ('集体', 3), ('领学团', 3), ('导师', 3), ('成长', 2), ('暑期', 2), ('教师', 2), ('班主任', 2), ('年级', 2), ('实践', 2), ('交流会', 2), ('心理', 2), ('简历', 2), ('年度', 2), ('秋季', 2), ('学期

In [42]:
# 查询 tag 的频次
def get_tag_times(dict_value_tags):
    tag_with_time = {}
    for key in dict_value_tags:
        for tag in dict_value_tags[key]:
            if tag not in tag_with_time:
                tag_with_time[tag] = 1
            else:
                tag_with_time[tag] += 1
    return tag_with_time


# 获取 tags 列表
tags_list = list(get_tag_times(title_with_tags).keys())
print(len(tags_list))
print(tags_list)

29
['学生工作', '培训会', '教学建设', '研讨会', '迎新', '师生交流', '会议', '主题教育', '班级活动', '就业发展', '校园生活', '党团活动', '社会实践', '调研活动', '交流会', '奖学金', '宣讲会', '其他活动', '文体活动', '赛事', '仪式', '学术', '讲座', '趣味活动', '答辩', '生活会', '采访', '课程学习', '汇报/报告']


In [43]:
# 构建词语数据矩阵
dist = np.zeros((len(title_with_tags), len(set(total_words))))
print(dist.shape)

(101, 334)


In [44]:
# 将数据填入矩阵
total_words_list = list(set(total_words))
title_list = list(title_with_words.keys())
for i in range(len(title_list)):
    print(title_with_words[title_list[i]])
    for word in title_with_words[title_list[i]]:
        for j in range(len(total_words_list)):
            if word == total_words_list[j]:
                print(word)
                dist[i][j] += 1

['学生', '骨干', '成长', '训练营', '举办', '暑期', '专题', '培训']
学生
骨干
成长
训练营
举办
暑期
专题
培训
['新生', '教师', '班主任', '工作', '研讨会', '召开']
新生
教师
班主任
工作
研讨会
召开
['级', '本科生', '线上', '新生', '家长会']
级
本科生
线上
新生
家长会
['级', '研究生', '新生', '入学', '教育', '活动']
级
研究生
新生
入学
教育
活动
['研究生班', '级新生', '见面会', '顺利', '举行']
研究生班
级新生
见面会
顺利
举行
['级', '就业', '动员会', '暨', '年级', '大会']
级
就业
动员会
暨
年级
大会
['年', '红色', '领航员', '党员', '舍导进', '宿舍', '活动']
年
红色
领航员
党员
舍导进
宿舍
活动
['赴', '湖北', '稳健', '医疗', '集团', '有限公司', '开展', '暑期', '实践', '调研', '活动']
赴
湖北
稳健
医疗
集团
有限公司
开展
暑期
实践
调研
活动
['年', '暑期社会', '实践', '分享', '交流会']
年
暑期社会
实践
分享
交流会
['年', '新生', '安全', '心理', '主题', '教育']
年
新生
安全
心理
主题
教育
['国奖', '国励', '宣讲会', '顺利', '举办']
国奖
国励
宣讲会
顺利
举办
['简历', '指导', '对', '简历', '修改', '活动', '成功', '举办']
简历
指导
对
简历
修改
活动
成功
举办
['年度', '秋季', '学期', '教职工', '大会', '举行']
年度
秋季
学期
教职工
大会
举行
['新生', '杯', '足球赛']
新生
杯
足球赛
['华中科技大学', '级', '开学典礼', '暨', '入学', '导航']
华中科技大学
级
开学典礼
暨
入学
导航
['华中', '大', '年', '秋季', '学期', '教师', '教学', '竞赛', '赛前', '培训']
华中
大
年
秋季
学期
教师
教学
竞赛
赛前
培训
['喻园', '管理', '论坛', '|', '香港城市大学

In [45]:
# 转化为 pd.DataFrame
data_words = pd.DataFrame(dist)
print(total_words_list)

['级', '营销', '就业', '孙新波', '党团', '管理学', '学硕', '竞赛', '矢田勝俊', '报告', '乒乓球赛', '劲牌', '副', '医疗', '项目', '东北大学', '发展', '英国', '精神', '医', '党', '对', '工商管理学', '管工', '来', '路', '对象', '片', '电脑', '杨治', '实践', '教职工', '交流会', '工程', '线上', '华中科技大学', '成长', '研究员', '管院', '乒乓球队', '导师', '终端产品', '课堂', '老师', '指导', '团', '旸', '领航员', '年级', '会计专业', '会', '论坛', '教学', '心理', '班会', '读书', '会计系', '八强', '专题学习', '九月', '重阳节', '建桥', '工商', '我院', '系列', '义务', '爱', '分享', '啦', '精彩', '揭牌', '副教授', '振兴', '宣讲', '舍导', '国际交流', '认证工作', '经验', '科', '华工', '第六', '信管', '思', '金融', '讲座', '当时', '会议', '选调', '开', '工商管理', '招新', '吧', '龙舟赛', '保研', '高端', '光电', '举行', '联合', '座谈会', '专访', '周年', '华中', '财政金融', '新加坡', '万得', '召开', '廖', '领跑', '二十大', '管理', '创新', '领学团', '专题', '见面会', '例会', '.', '文化', '进', '喻园', '评选', '年度', '月', '逐梦正', '融合', '研究', '调研', '班班', '迎新晚会', '第一次', '传统节日', '第三轮', '顺利', '党日', '集体', '我为人人', '杯', '大', '决赛', '助理', '开幕式', '湖北', '管理科学', '奖学金', '拍', '修改', '赴', '新闻宣传', '教室', '答谢', '暑期', '汇报', '朱同伟', '和', '党史', '专业', '入学', '研工组', '家长会', '国奖', '企业', '党课

In [46]:
# tag 矩阵构建
# 定义 0 矩阵
dist_matrix = np.zeros((len(title_with_tags), len(get_tag_times(title_with_tags)),))
# 获取 tags 列表
tags_list = list(get_tag_times(title_with_tags).keys())
# 获取 title 的 list
title_list = list(title_with_tags.keys())
# 根据 tag 生成矩阵
for mission_index in range(len(title_with_tags)):
    for tag in title_with_tags[title_list[mission_index]]:
        for tag_index in range(len(tags_list)):
            if tag == tags_list[tag_index]:
                dist_matrix[mission_index][tag_index] = 1

data_label = pd.DataFrame(dist_matrix)

In [71]:
# 分割数据为训练集与测试集
X_train, X_test, y_train, y_test = train_test_split(data_words, data_label, test_size=0.1, random_state=0)

In [72]:
# 模型定义
def deep_model(feature_dim, label_dim):
    model = Sequential()
    print("create model. feature_dim ={}, label_dim ={}".format(feature_dim, label_dim))
    # model.add(Embedding(5, 32, input_length=79))
    # model.add(Flatten())
    model.add(Dense(512, activation='relu', input_dim=feature_dim))
    model.add(Dense(label_dim, activation='softmax'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [73]:
# 模型训练函数
def train_deep(X_train, y_train, X_test, y_test, epochs):
    feature_dim = X_train.shape[1]
    label_dim = y_train.shape[1]
    model = deep_model(feature_dim, label_dim)
    model.summary()
    model.fit(X_train, y_train, batch_size=16, epochs=epochs, validation_data=(X_test, y_test))
    return model

In [76]:
# 开始训练
my_model = train_deep(X_train, y_train, X_test, y_test, 20)

create model. feature_dim =334, label_dim =29
Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 512)               171520    
                                                                 
 dense_39 (Dense)            (None, 29)                14877     
                                                                 
Total params: 186,397
Trainable params: 186,397
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
6/6 [==============================] - 1s 34ms/step - loss: 0.6572 - accuracy: 0.0222 - val_loss: 0.6037 - val_accuracy: 0.0909
Epoch 2/20
6/6 [==============================] - 0s 9ms/step - loss: 0.5674 - accuracy: 0.1111 - val_loss: 0.5243 - val_accuracy: 0.0909
Epoch 3/20
6/6 [==============================] - 0s 9ms/step - loss: 0.4783 - accuracy: 0.2333 - val_loss: 0.4475 - val